In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import (cross_val_score, train_test_split,
                                    GridSearchCV, RandomizedSearchCV)
from sklearn.preprocessing import Imputer
%load_ext autoreload
%autoreload 2

In [2]:
train = pd.read_csv('data/train_final.csv')
test = pd.read_csv('data/test_final.csv')
train = train.drop(['id'], axis=1)
test = test.drop(['id'], axis=1)

In [3]:
# Divide dataset into X and y
y = train.Y
X = train.drop(["Y"], axis=1)
X_test = test

In [4]:
# Impute missing features
from sklearn.preprocessing import Imputer

# train = train.apply(lambda x: x.fillna(x.value_counts().index[0]))
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
train_xform = imp.fit_transform(X)
# TODO: Impute dataframe so that F5 uses median
# and F19 uses mean. For now, we'll impute via mean for both.

X = pd.DataFrame(train_xform, columns=X.columns)
test_xform = imp.transform(X_test)
X_test = pd.DataFrame(test_xform, columns=X_test.columns)

X.head(5)

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27
0,1.0,0.0,0.107576,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,104.0,22902.0,1.0,0.0,18.0,0.042295,1.0,0.0,27.0,0.028250
1,1.0,0.0,0.142357,0.0,0.0,7.0,1.0,1.0,1.0,1.0,...,144.0,11400.0,1.0,0.0,8.0,0.021417,1.0,0.0,67.0,0.253574
2,1.0,0.0,0.492318,0.0,3.0,4205.0,1.0,1.0,3.0,1.0,...,112.0,4833.0,1.0,0.0,13.0,0.502212,1.0,1.0,35.0,0.373397
3,1.0,0.0,-0.053028,0.0,2.0,2.0,1.0,1.0,5.0,2.0,...,127.0,3250.0,1.0,1.0,8.0,0.000000,1.0,0.0,50.0,0.674254
4,1.0,0.0,0.730797,0.0,0.0,11.0,1.0,1.0,1.0,1.0,...,148.0,4000.0,1.0,1.0,5.0,0.787592,1.0,0.0,71.0,0.371157


In [5]:
# Split data.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=42)

In [6]:
import os
os.environ['OMP_NUM_THREADS'] = '1' # Let OpenMP use 1 thread to avoid possible subprocess call hangs
import hpsklearn

/home/aetherzephyr/anaconda3/envs/datasci/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Let's do some hyperparameter searching with Hyperopt. This may be a better choice than what we were doing previously - random sampling of polynomial combinations of features.

In [16]:
print(X_train.shape)
print(y_train.shape)

(44998, 27)
(44998,)


In [19]:
from hpsklearn import HyperoptEstimator, random_forest
from hyperopt import tpe
import hyperopt


hour = 3600  # Number of seconds in an hour
# Create the estimator object
estim = HyperoptEstimator(algo=tpe.suggest,
                           classifier=random_forest('rf'),
                           max_evals=100,
                           preprocessing=[], # Have to set preprocessing to [] to avoid error
                           trial_timeout=2 * hour)

# Search the space of classifiers and preprocessing steps and their
# respective hyperparameters in sklearn to fit a model to the data
estim.fit(X_train, y_train, n_folds=10)

# Make a prediction using the optimized model
prediction = estim.predict(X_val)

# Report the accuracy of the classifier on a given set of data
score = estim.score(X_val, y_val)
print(score)

# Return instances of the classifier and preprocessing steps
model = estim.best_model()
print(model)

IndexError: indices are out-of-bounds